##### This notebook contains code to conduct commonality analysis

In [18]:
#load libraries
import pandas as pd
import numpy as np
import glob
import os

from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from scipy.stats import linregress
from sklearn.linear_model import RidgeCV

In [19]:
#function return r2 from linear regression between feature(s) and target
def get_r2(X, y):
    if len(X.shape)<2:
        r2 = LinearRegression().fit(StandardScaler().fit_transform(X.values.reshape(-1, 1)), y).score(StandardScaler().fit_transform(X.values.reshape(-1, 1)), y)
    else:
        r2 = LinearRegression().fit(StandardScaler().fit_transform(X.values), y).score(StandardScaler().fit_transform(X.values), y)
    return r2
        

In [38]:
#function return r2 from Ridge regression between feature(s) and target
def get_r2_ridge(X, y):
    
    alphas=np.logspace(-1, 2, 70) #from 0.1 to 100
    
    ridge = RidgeCV(alphas=alphas, scoring='r2', cv=5)
    
    r2 = ridge.fit(StandardScaler().fit_transform(X.values), y).score(StandardScaler().fit_transform(X.values), y)
    
    balpha=ridge.alpha_
    
    return r2, balpha
        

In [21]:
#path to output directory 
path = '/media/data/HCPAging/data/MLTablesMultCon/output_5cv_sexAdj_noStdTarg_STDstackFeatures/'

In [22]:
#load dataframes

#age prediction from test
df_age = pd.read_csv(path+'output_5cv_sexAdj_noStdTarg_STDstackFeatures_interview_age/all_fold_y-pred_y-obs.csv', index_col=0)
#age prediction from train
df_age_train = pd.read_csv(path+'output_5cv_sexAdj_noStdTarg_STDstackFeatures_interview_age/all_fold_y-pred_y-obs_train.csv', index_col=0)
#fluid cognition prediction
df_cog = pd.read_csv(path+'output_5cv_sexAdj_noStdTarg_STDstackFeatures_nih_fluidcogcomp_unadjusted/all_fold_y-pred_y-obs.csv', index_col=0)

In [23]:
#define original targets and predicted values for each type
cog_orig = df_cog['y_real']
cog_pred = df_cog.iloc[:, :-2]

age_orig = df_age['y_real']
age_pred = df_age.iloc[:, :-2]

In [24]:
#calculate brain age corrected

dct_corrected ={}

for fold in sorted(set(df_age_train['fold'])):
    dct_corrected[fold] ={}
    
    df_pred = df_age_train[df_age_train['fold']==fold].iloc[:, :-2]
    df_orig = df_age_train[df_age_train['fold']==fold]['y_real']
    df_test = df_age[df_age['fold']==fold].iloc[:, :-2]
    
    for col in df_pred.columns:
        slope, intercept, _, _, _, = linregress(df_orig, df_pred[col])
        
        out = (df_test[col] - intercept)/slope
        
        
        dct_corrected[fold][col]= out
        
age_pred_cor = pd.DataFrame(dct_corrected[0])
for fold in sorted(set(df_age_train['fold']))[1:]:
    age_pred_cor = pd.concat([age_pred_cor, pd.DataFrame(dct_corrected[fold])], axis=0)

In [25]:
#calculate brain age gap, usual and corrected

age_gap = pd.DataFrame()
for col in age_pred.columns:
    age_gap[col] = age_orig - age_pred[col]


age_gap_cor = pd.DataFrame()
for col in age_pred_cor.columns:
    age_gap_cor[col] = age_orig - age_pred_cor[col]

In [26]:
# one predictor
#cognition                              cog_orig        1
#age                                    age_orig        1
#brain age                              age_pred        26
#brain age gap (delta)                  age_gap         26
#corrected brain age                    age_pred_cor    26
#corrected brain age gap (delta)        age_gap_cor     26

In [27]:
#Predicting cognition with age

#one predictor (one of brain age indexes)

dct_r2_one = {}
dct_corR_one = {}
dct_corPval_one = {}

for dset, dset_name in zip([age_orig, age_pred, age_gap, age_pred_cor, age_gap_cor], 
                           ['age_orig', 'age_pred', 'age_gap', 'age_pred_cor', 'age_gap_cor']):  
    #
    
    if len(dset.shape)<2:
        
        dct_r2_one[dset_name] = get_r2(dset, cog_orig)
        
    else:
        
        dct_r2_one[dset_name] = {}
        dct_corR_one[dset_name] = {}
        dct_corPval_one[dset_name] = {}
        
        for col in dset.columns:
            
            dct_r2_one[dset_name][col] = get_r2(dset[col], cog_orig)
            dct_corR_one[dset_name][col], dct_corPval_one[dset_name][col] = pearsonr(dset[col], cog_orig)
        
        dct_r2_one[dset_name] = pd.Series(dct_r2_one[dset_name])
        dct_corR_one[dset_name] = pd.Series(dct_corR_one[dset_name])
        dct_corPval_one[dset_name] = pd.Series(dct_corPval_one[dset_name])

print('Predicting cognition with age')
print(' ')
print('age_orig', dct_r2_one['age_orig'])
print('age_orig correlation with cog', pearsonr(age_orig, cog_orig))

df_r2_one = pd.DataFrame()
df_corR_one = pd.DataFrame()
df_corPval_one = pd.DataFrame()

for key in list(dct_r2_one.keys())[1:]:
    df_r2_one['r2_1_'+key] = dct_r2_one[key]
    df_corR_one['R_1_'+key] = dct_corR_one[key]
    df_corPval_one['Pval_1_'+key] = dct_corPval_one[key]

print(' ')    
print('R2')    
display(df_r2_one.round(6))
print(' ') 
print('Correlation')
display(df_corR_one.round(6))
print(' ') 
print('Pvalue for correlation')
display(df_corPval_one.round(6))

Predicting cognition with age
 
age_orig 0.32073185105582647
age_orig correlation with cog PearsonRResult(statistic=-0.5663319265729474, pvalue=4.353600662708515e-44)
 
R2


,r2_1_age_pred,r2_1_age_gap,r2_1_age_pred_cor,r2_1_age_gap_cor
carit1,0.052275,0.278114,0.056524,0.036141
carit3,0.101934,0.228756,0.105562,0.028827
carit4,0.051659,0.267819,0.052987,0.022561
face1,0.118230,0.199998,0.114327,0.012302
face2,0.115699,0.199401,0.118205,0.014101
face3,0.113068,0.223987,0.115074,0.006080
face4,0.043832,0.272638,0.044384,0.026787
face5,0.166395,0.165657,0.166601,0.007732
face6,0.175158,0.166654,0.166113,0.002471
vism,0.129792,0.195600,0.132140,0.012664


 
Correlation


,R_1_age_pred,R_1_age_gap,R_1_age_pred_cor,R_1_age_gap_cor
carit1,-0.228638,-0.527366,-0.237748,-0.190108
carit3,-0.319272,-0.478285,-0.324903,-0.169785
carit4,-0.227287,-0.517513,-0.230190,-0.150203
face1,-0.343845,-0.447212,-0.338122,-0.110916
face2,-0.340146,-0.446543,-0.343810,-0.118749
face3,-0.336255,-0.473272,-0.339226,-0.077971
face4,-0.209361,-0.522147,-0.210674,-0.163668
face5,-0.407916,-0.407010,-0.408168,-0.087930
face6,-0.418519,-0.408233,-0.407569,-0.049710
vism,-0.360266,-0.442267,-0.363511,-0.112533


 
Pvalue for correlation


,Pval_1_age_pred,Pval_1_age_gap,Pval_1_age_pred_cor,Pval_1_age_gap_cor
carit1,0.000000,0.0,0.000000,0.000017
carit3,0.000000,0.0,0.000000,0.000128
carit4,0.000000,0.0,0.000000,0.000717
face1,0.000000,0.0,0.000000,0.012718
face2,0.000000,0.0,0.000000,0.007613
face3,0.000000,0.0,0.000000,0.080331
face4,0.000002,0.0,0.000002,0.000224
face5,0.000000,0.0,0.000000,0.048503
face6,0.000000,0.0,0.000000,0.265320
vism,0.000000,0.0,0.000000,0.011468


In [39]:
#Predicting cognition with age and predicted age index

#two predictors (one of brain age indexes and real age)

dct_r2_two = {}
dct_u1_two = {}
dct_u2_two = {}
dct_c_two = {}

dct_alp_two = {} ###

for dset, dset_name in zip([age_pred, age_gap, age_pred_cor, age_gap_cor], 
                           ['age_pred', 'age_gap', 'age_pred_cor', 'age_gap_cor']):  
    #
    dct_r2_two[dset_name] = {}
    dct_u1_two[dset_name] = {}
    dct_u2_two[dset_name] = {}
    dct_c_two[dset_name] = {}
    
    dct_alp_two[dset_name] = {} ###
    
    for col in dset.columns:
        #
        r2_i = get_r2(dset[col], cog_orig)
        r2_j = get_r2(age_orig, cog_orig)
        r2_ij = get_r2_ridge(pd.concat([dset[col], age_orig], axis=1), cog_orig)[0]
        
        alp_ij = get_r2_ridge(pd.concat([dset[col], age_orig], axis=1), cog_orig)[1] ###

        u_i = r2_ij - r2_j
        u_j = r2_ij - r2_i

        c_ij = r2_ij - u_i - u_j
        
        dct_r2_two[dset_name][col] = r2_ij
        dct_u1_two[dset_name][col] = u_i
        dct_u2_two[dset_name][col] = u_j
        dct_c_two[dset_name][col] = c_ij
        
        dct_alp_two[dset_name][col] = alp_ij ###
        
    dct_r2_two[dset_name] = pd.Series(dct_r2_two[dset_name])
    dct_u1_two[dset_name] = pd.Series(dct_u1_two[dset_name])
    dct_u2_two[dset_name] = pd.Series(dct_u2_two[dset_name])
    dct_c_two[dset_name] = pd.Series(dct_c_two[dset_name])
    
    dct_alp_two[dset_name] = pd.Series(dct_alp_two[dset_name]) ###


df_r2_two = pd.DataFrame(dct_r2_two)
df_r2_two.columns = ['r2_2_'+i+"+age-orig" for i in df_r2_two.columns]

df_alp_two = pd.DataFrame(dct_alp_two) ###
df_alp_two.columns = ['Alpha2_'+i+"+age-orig" for i in df_alp_two.columns] ###

df_u1_two = pd.DataFrame(dct_u1_two)
df_u1_two.columns = ['U_2_'+i for i in df_u1_two.columns]

df_u2_two = pd.DataFrame(dct_u2_two)
df_u2_two.columns = ['U_2_age-orig_('+i+')' for i in df_u2_two.columns]

df_c_two = pd.DataFrame(dct_c_two)
df_c_two.columns = ['C_2_'+i+'+age-orig' for i in df_c_two.columns]

print('Two predictors for cognition prediction')
print('R2, uniq R2, common R2')
display(pd.concat([df_r2_two, df_u1_two, df_u2_two, df_c_two], axis=1) )

Two predictors for cognition prediction
R2, uniq R2, common R2


,r2_2_age_pred+age-orig,r2_2_age_gap+age-orig,r2_2_age_pred_cor+age-orig,r2_2_age_gap_cor+age-orig,U_2_age_pred,U_2_age_gap,U_2_age_pred_cor,U_2_age_gap_cor,U_2_age-orig_(age_pred),U_2_age-orig_(age_gap),U_2_age-orig_(age_pred_cor),U_2_age-orig_(age_gap_cor),C_2_age_pred+age-orig,C_2_age_gap+age-orig,C_2_age_pred_cor+age-orig,C_2_age_gap_cor+age-orig
carit1,0.323504,0.323556,0.323504,0.323567,0.002773,0.002824,2.772634e-03,0.002835,0.271229,0.045442,0.266980,0.287426,0.049503,0.275290,0.053751,0.033306
carit3,0.322477,0.322116,0.322381,0.322226,0.001745,0.001385,1.648665e-03,0.001494,0.220543,0.093360,0.216819,0.293399,0.100189,0.227372,0.103913,0.027333
carit4,0.320805,0.320842,0.320828,0.320848,0.000073,0.000110,9.568626e-05,0.000116,0.269146,0.053022,0.267840,0.298287,0.051586,0.267709,0.052892,0.022445
face1,0.324703,0.324649,0.323413,0.323282,0.003971,0.003917,2.681526e-03,0.002550,0.206473,0.124651,0.209087,0.310979,0.114259,0.196081,0.111645,0.009753
face2,0.322428,0.321556,0.322564,0.322470,0.001696,0.000824,1.831929e-03,0.001738,0.206729,0.122155,0.204359,0.308368,0.114003,0.198577,0.116373,0.012364
face3,0.328923,0.328849,0.329402,0.329402,0.008191,0.008117,8.670180e-03,0.008670,0.215855,0.104862,0.214328,0.323322,0.104877,0.215870,0.106404,-0.002590
face4,0.320650,0.318856,0.320928,0.320779,-0.000082,-0.001876,1.960388e-04,0.000047,0.276818,0.046218,0.276544,0.293992,0.043914,0.274514,0.044188,0.026740
face5,0.328293,0.328255,0.327611,0.327530,0.007562,0.007524,6.878995e-03,0.006798,0.161898,0.162598,0.161010,0.319798,0.158834,0.158134,0.159722,0.000934
face6,0.336803,0.336830,0.332384,0.332100,0.016071,0.016098,1.165233e-02,0.011368,0.161645,0.170176,0.166271,0.329629,0.159087,0.150556,0.154461,-0.008897
vism,0.325687,0.325671,0.325460,0.325450,0.004955,0.004939,4.728566e-03,0.004718,0.195896,0.130071,0.193320,0.312786,0.124836,0.190661,0.127412,0.007946


In [40]:
df_alp_two

,Alpha2_age_pred+age-orig,Alpha2_age_gap+age-orig,Alpha2_age_pred_cor+age-orig,Alpha2_age_gap_cor+age-orig
carit1,10.000000,1.350314,9.047357,4.489251
carit3,5.484417,7.405685,4.961948,11.052951
carit4,6.700188,1.649648,6.700188,5.484417
face1,13.503140,6.061899,12.216773,14.924955
face2,10.000000,14.924955,11.052951,13.503140
face3,5.484417,2.462092,5.484417,4.489251
face4,11.052951,27.213388,9.047357,14.924955
face5,8.185467,4.061586,8.185467,10.000000
face6,5.484417,0.332460,12.216773,16.496481
vism,4.489251,2.227543,5.484417,5.484417


In [43]:
#Predicting cognition with age, predicted age index and predicted cognition

#three predictors (one of brain age indexes, real age, predicted cognition)

dct_r2_one_cog = {}
dct_r2_two_age_cogpr = {}
dct_r2_two_agepr_cogpr = {}
dct_r2_three = {}

dct_u1_three = {}
dct_u2_three = {}
dct_u3_three = {}

dct_c1_three = {}
dct_c2_three = {}
dct_c3_three = {}
dct_c4_three = {}

dct_alp_two_age_cogpr = {} ###
dct_alp_two_agepr_cogpr = {} ###
dct_alp_three = {} ###

for dset, dset_name in zip([age_pred, age_gap, age_pred_cor, age_gap_cor], 
                           ['age_pred', 'age_gap', 'age_pred_cor', 'age_gap_cor']):  
    #
    dct_r2_one_cog[dset_name] = {}
    dct_r2_two_age_cogpr[dset_name] = {}
    dct_r2_two_agepr_cogpr[dset_name] = {}
    dct_r2_three[dset_name] = {}

    dct_u1_three[dset_name] = {}
    dct_u2_three[dset_name] = {}
    dct_u3_three[dset_name] = {}

    dct_c1_three[dset_name] = {}
    dct_c2_three[dset_name] = {}
    dct_c3_three[dset_name] = {}
    dct_c4_three[dset_name] = {}
    
    dct_alp_two_age_cogpr[dset_name] = {} ###
    dct_alp_two_agepr_cogpr[dset_name] = {} ###
    dct_alp_three[dset_name] = {}    ###
    
    for col in dset.columns:
        #
        r2_i = get_r2( dset[col], cog_orig)
        r2_j = get_r2( age_orig, cog_orig)
        r2_k = get_r2( cog_pred[col], cog_orig) #

        r2_jk = get_r2_ridge( pd.concat([ age_orig, cog_pred[col]],axis=1), cog_orig)[0] #
        r2_ik = get_r2_ridge( pd.concat([ dset[col], cog_pred[col]],axis=1), cog_orig)[0] #
        r2_ij = get_r2_ridge( pd.concat([ dset[col], age_orig],axis=1), cog_orig)[0]

        r2_ijk = get_r2_ridge( pd.concat([ dset[col], age_orig, cog_pred[col] ],axis=1), cog_orig)[0] #
        
        alp_jk = get_r2_ridge( pd.concat([ age_orig, cog_pred[col]],axis=1), cog_orig)[1] ###
        alp_ik = get_r2_ridge( pd.concat([ dset[col], cog_pred[col]],axis=1), cog_orig)[1] ###
        alp_ij = get_r2_ridge( pd.concat([ dset[col], age_orig],axis=1), cog_orig)[1] ###

        alp_ijk = get_r2_ridge( pd.concat([ dset[col], age_orig, cog_pred[col] ],axis=1), cog_orig)[1] ###

        u_i = r2_ijk - r2_jk # uniq for age_predicted
        u_j = r2_ijk - r2_ik # uniq for age
        u_k = r2_ijk - r2_ij # uniq for cog predicted

        c_ij = r2_ik + r2_jk - r2_k - r2_ijk
        c_ik = r2_ij + r2_jk - r2_j - r2_ijk
        c_jk = r2_ij + r2_ik - r2_i - r2_ijk
        c_ijk = r2_i + r2_j + r2_k - r2_ij - r2_ik - r2_jk + r2_ijk
        
        
        dct_r2_one_cog[dset_name][col] = r2_k
        dct_r2_two_age_cogpr[dset_name][col] = r2_jk
        dct_r2_two_agepr_cogpr[dset_name][col] = r2_ik
        dct_r2_three[dset_name][col] = r2_ijk

        dct_u1_three[dset_name][col] = u_i
        dct_u2_three[dset_name][col] = u_j
        dct_u3_three[dset_name][col] = u_k

        dct_c1_three[dset_name][col] = c_ij
        dct_c2_three[dset_name][col] = c_ik
        dct_c3_three[dset_name][col] = c_jk
        dct_c4_three[dset_name][col] = c_ijk
        
        dct_alp_two_age_cogpr[dset_name][col] = alp_jk ###
        dct_alp_two_agepr_cogpr[dset_name][col] = alp_ik ###
        dct_alp_three[dset_name][col] = alp_ijk ###
    
    dct_r2_one_cog[dset_name] = pd.Series(dct_r2_one_cog[dset_name])
    dct_r2_two_age_cogpr[dset_name] = pd.Series(dct_r2_two_age_cogpr[dset_name])
    dct_r2_two_agepr_cogpr[dset_name] = pd.Series(dct_r2_two_agepr_cogpr[dset_name])
    dct_r2_three[dset_name] = pd.Series(dct_r2_three[dset_name])

    dct_u1_three[dset_name] = pd.Series(dct_u1_three[dset_name])
    dct_u2_three[dset_name] = pd.Series(dct_u2_three[dset_name])
    dct_u3_three[dset_name] = pd.Series(dct_u3_three[dset_name])

    dct_c1_three[dset_name] = pd.Series(dct_c1_three[dset_name])
    dct_c2_three[dset_name] = pd.Series(dct_c2_three[dset_name])
    dct_c3_three[dset_name] = pd.Series(dct_c3_three[dset_name])
    dct_c4_three[dset_name] = pd.Series(dct_c4_three[dset_name])
    
    dct_alp_two_age_cogpr[dset_name] = pd.Series(dct_alp_two_age_cogpr[dset_name]) ###
    dct_alp_two_agepr_cogpr[dset_name] = pd.Series(dct_alp_two_agepr_cogpr[dset_name]) ###
    dct_alp_three[dset_name] = pd.Series(dct_alp_three[dset_name])    ###

df_r2_one_cog = pd.DataFrame(dct_r2_one_cog['age_pred'])
df_r2_one_cog.columns = ['r2_1_cog_pred']
df_r2_two_age_cogpr = pd.DataFrame(dct_r2_two_age_cogpr['age_pred'])
df_r2_two_age_cogpr.columns = ['r2_3_2_age-orig+cog-pred']
df_r2_two_agepr_cogpr = pd.DataFrame(dct_r2_two_agepr_cogpr)
df_r2_two_agepr_cogpr.columns = ['r2_3_2_'+i+'+cog-pred' for i in df_r2_two_agepr_cogpr.columns]
df_r2_three = pd.DataFrame(dct_r2_three)
df_r2_three.columns = ['r2_3_'+i+'+age+cog-pred' for i in df_r2_three.columns]

df_alp_two_age_cogpr = pd.DataFrame(dct_alp_two_age_cogpr['age_pred']) ###
df_alp_two_age_cogpr.columns = ['Alpha3_2_age-orig+cog-pred'] ###
df_alp_two_agepr_cogpr = pd.DataFrame(dct_alp_two_agepr_cogpr) ###
df_alp_two_agepr_cogpr.columns = ['Alpha3_2_'+i+'+cog-pred' for i in df_alp_two_agepr_cogpr.columns] ###
df_alp_three = pd.DataFrame(dct_alp_three) ###
df_alp_three.columns = ['Alpha3_3_'+i+'+age+cog-pred' for i in df_alp_three.columns] ###

df_u1_three = pd.DataFrame(dct_u1_three)
df_u1_three.columns = ['U_3_'+i for i in df_u1_three.columns]
df_u2_three = pd.DataFrame(dct_u2_three)
df_u2_three.columns = ['U_3_age-orig_('+i+')' for i in df_u2_three.columns]
df_u3_three = pd.DataFrame(dct_u3_three)
df_u3_three.columns = ['U_3_cog-pred_('+i+')' for i in df_u3_three.columns]

df_c1_three = pd.DataFrame(dct_c1_three)
df_c1_three.columns = ['C_3_2_'+i+'+age-orig' for i in df_c1_three.columns]
df_c2_three = pd.DataFrame(dct_c2_three)
df_c2_three.columns = ['C_3_2_'+i+'+cog-pred' for i in df_c2_three.columns]
df_c3_three = pd.DataFrame(dct_c3_three)
df_c3_three.columns = ['C_3_2_age-orig+cog-pred_('+i+')' for i in df_c3_three.columns]
df_c4_three = pd.DataFrame(dct_c4_three)
df_c4_three.columns = ['C_3_'+i+'+age+cog-pred' for i in df_c4_three.columns]


print('Three predictors for cognition prediction')
print('R2, uniq R2, common R2')
display(pd.concat([df_r2_one_cog, df_r2_two_age_cogpr, df_r2_two_agepr_cogpr, df_r2_three,
                   df_u1_three, df_u2_three, df_u3_three, 
                   df_c1_three, df_c2_three, df_c3_three, df_c4_three], axis=1))

Three predictors for cognition prediction
R2, uniq R2, common R2


,r2_1_cog_pred,r2_3_2_age-orig+cog-pred,r2_3_2_age_pred+cog-pred,r2_3_2_age_gap+cog-pred,r2_3_2_age_pred_cor+cog-pred,r2_3_2_age_gap_cor+cog-pred,r2_3_age_pred+age+cog-pred,r2_3_age_gap+age+cog-pred,r2_3_age_pred_cor+age+cog-pred,r2_3_age_gap_cor+age+cog-pred,...,C_3_2_age_pred_cor+cog-pred,C_3_2_age_gap_cor+cog-pred,C_3_2_age-orig+cog-pred_(age_pred),C_3_2_age-orig+cog-pred_(age_gap),C_3_2_age-orig+cog-pred_(age_pred_cor),C_3_2_age-orig+cog-pred_(age_gap_cor),C_3_age_pred+age+cog-pred,C_3_age_gap+age+cog-pred,C_3_age_pred_cor+age+cog-pred,C_3_age_gap_cor+age+cog-pred
carit1,0.031342,0.324604,0.053327,0.304024,0.057089,0.117597,0.324717,0.324660,0.324808,0.325011,...,0.002568,0.002428,-0.000162,0.024806,-0.000739,0.080011,0.027632,0.002664,0.028209,-0.052542
carit3,0.090492,0.328717,0.112184,0.302937,0.114791,0.188962,0.328668,0.325608,0.328677,0.328334,...,0.001688,0.001877,0.004059,0.070690,0.002932,0.154027,0.078448,0.011817,0.079574,-0.071521
carit4,0.058000,0.327561,0.067414,0.313592,0.068720,0.135274,0.329269,0.328751,0.329060,0.329053,...,-0.001403,-0.001376,0.007290,0.037864,0.007500,0.104508,0.043880,0.013307,0.043671,-0.053337
face1,0.171927,0.364009,0.172988,0.365253,0.171863,0.287370,0.371282,0.371352,0.374679,0.373947,...,-0.007988,-0.007388,0.008179,0.118551,0.006271,0.224403,0.120470,0.010098,0.122378,-0.095754
face2,0.182768,0.365989,0.180659,0.379755,0.181291,0.309362,0.383209,0.381753,0.382506,0.381821,...,-0.014686,-0.014095,0.004179,0.120157,0.003143,0.235909,0.133332,0.017354,0.134368,-0.098398
face3,0.192793,0.388305,0.194155,0.397329,0.194631,0.301533,0.399261,0.398871,0.398122,0.397985,...,-0.001146,-0.001010,0.010749,0.103320,0.010837,0.226870,0.114471,0.021900,0.114383,-0.101650
face4,0.077471,0.332574,0.075954,0.341568,0.076801,0.193342,0.344314,0.342705,0.344126,0.344050,...,-0.011356,-0.011429,0.008458,0.045081,0.009219,0.143284,0.057171,0.020548,0.056409,-0.077655
face5,0.242093,0.390747,0.246128,0.398775,0.246161,0.349042,0.403562,0.403108,0.404882,0.404701,...,-0.007255,-0.007156,0.004464,0.158266,0.002289,0.264139,0.167613,0.013812,0.169788,-0.092062
face6,0.244930,0.400847,0.250884,0.401641,0.249255,0.342572,0.408069,0.407904,0.411520,0.411443,...,0.000979,0.000772,0.004460,0.163913,0.004006,0.260757,0.160355,0.000902,0.160809,-0.095943
vism,0.166127,0.365524,0.174748,0.366211,0.175781,0.283905,0.374408,0.374209,0.374723,0.374636,...,-0.004470,-0.004394,-0.003764,0.122073,-0.005622,0.222055,0.125099,-0.000738,0.126957,-0.100720


In [44]:
df_alp_three

,Alpha3_3_age_pred+age+cog-pred,Alpha3_3_age_gap+age+cog-pred,Alpha3_3_age_pred_cor+age+cog-pred,Alpha3_3_age_gap_cor+age+cog-pred
carit1,18.233480,4.489251,16.496481,8.185467
carit3,33.245979,90.473572,33.245979,44.892513
carit4,10.000000,27.213388,8.185467,10.000000
face1,27.213388,60.618990,20.153377,49.619476
face2,4.961948,40.615860,6.061899,30.078825
face3,4.061586,16.496481,6.700188,16.496481
face4,20.153377,74.056847,27.213388,44.892513
face5,6.700188,24.620924,4.961948,16.496481
face6,2.721339,14.924955,0.100000,8.185467
vism,3.674662,20.153377,3.324598,12.216773


In [45]:
#add extra columns from ML

dct_age_brain={}
for col in age_pred.columns:
    dct_age_brain[col] = get_r2( age_pred[col], age_orig)
    
age_brain = pd.Series(dct_age_brain)
age_brain.name = 'r2_AGE_brain'

cog_brain = df_r2_one_cog['r2_1_cog_pred']
cog_brain.name = 'r2_COG_brain'

In [47]:
#assemble into one big table and save

big_table = pd.concat([cog_brain, age_brain, df_r2_one, 
                       df_r2_two, df_u1_two, df_u2_two, df_c_two,
                       df_r2_two_age_cogpr, df_r2_two_agepr_cogpr, df_r2_three,
                       df_u1_three, df_u2_three, df_u3_three, 
                       df_c1_three, df_c2_three, df_c3_three, df_c4_three], axis=1)
big_table.index = ['carit1-allNoGo-allGo', 'carit3-allGo', 'carit4-allNoGo', 'face1-encoding', 'face2-recall', 'face3-distractor', 'face4-encoding-recall',
                   'face5-encoding-distractor', 'face6-recall-distractor', 'VisualMotor', 'carit_FC', 'face_FC', 'vism_FC', 'cort_thickness',
                   'cort_surface', 'subcortex', 'Total_Brain_Volume', 'Resting_State_FC_75PC', 
                   'All-Modality_con+FC(set1)', 'Task_con(set2)', 'Non-Task(set3)', 
                   'Task_FC(set4)', 'Task_con+FC(set5)', 'All-modality_con(set6)', 
                   'All-modality_FC(set7)', 'FC_rest_task(set8)']
display(big_table)

#save
big_table.to_csv(path+'commonality_cog_fluid_new.csv')

,r2_COG_brain,r2_AGE_brain,r2_1_age_pred,r2_1_age_gap,r2_1_age_pred_cor,r2_1_age_gap_cor,r2_2_age_pred+age-orig,r2_2_age_gap+age-orig,r2_2_age_pred_cor+age-orig,r2_2_age_gap_cor+age-orig,...,C_3_2_age_pred_cor+cog-pred,C_3_2_age_gap_cor+cog-pred,C_3_2_age-orig+cog-pred_(age_pred),C_3_2_age-orig+cog-pred_(age_gap),C_3_2_age-orig+cog-pred_(age_pred_cor),C_3_2_age-orig+cog-pred_(age_gap_cor),C_3_age_pred+age+cog-pred,C_3_age_gap+age+cog-pred,C_3_age_pred_cor+age+cog-pred,C_3_age_gap_cor+age+cog-pred
carit1-allNoGo-allGo,0.031342,0.098319,0.052275,0.278114,0.056524,0.036141,0.323504,0.323556,0.323504,0.323567,...,0.002568,0.002428,-0.000162,0.024806,-0.000739,0.080011,0.027632,0.002664,0.028209,-0.052542
carit3-allGo,0.090492,0.249086,0.101934,0.228756,0.105562,0.028827,0.322477,0.322116,0.322381,0.322226,...,0.001688,0.001877,0.004059,0.070690,0.002932,0.154027,0.078448,0.011817,0.079574,-0.071521
carit4-allNoGo,0.058000,0.146194,0.051659,0.267819,0.052987,0.022561,0.320805,0.320842,0.320828,0.320848,...,-0.001403,-0.001376,0.007290,0.037864,0.007500,0.104508,0.043880,0.013307,0.043671,-0.053337
face1-encoding,0.171927,0.258833,0.118230,0.199998,0.114327,0.012302,0.324703,0.324649,0.323413,0.323282,...,-0.007988,-0.007388,0.008179,0.118551,0.006271,0.224403,0.120470,0.010098,0.122378,-0.095754
face2-recall,0.182768,0.288199,0.115699,0.199401,0.118205,0.014101,0.322428,0.321556,0.322564,0.322470,...,-0.014686,-0.014095,0.004179,0.120157,0.003143,0.235909,0.133332,0.017354,0.134368,-0.098398
face3-distractor,0.192793,0.203222,0.113068,0.223987,0.115074,0.006080,0.328923,0.328849,0.329402,0.329402,...,-0.001146,-0.001010,0.010749,0.103320,0.010837,0.226870,0.114471,0.021900,0.114383,-0.101650
face4-encoding-recall,0.077471,0.148571,0.043832,0.272638,0.044384,0.026787,0.320650,0.318856,0.320928,0.320779,...,-0.011356,-0.011429,0.008458,0.045081,0.009219,0.143284,0.057171,0.020548,0.056409,-0.077655
face5-encoding-distractor,0.242093,0.355701,0.166395,0.165657,0.166601,0.007732,0.328293,0.328255,0.327611,0.327530,...,-0.007255,-0.007156,0.004464,0.158266,0.002289,0.264139,0.167613,0.013812,0.169788,-0.092062
face6-recall-distractor,0.244930,0.304936,0.175158,0.166654,0.166113,0.002471,0.336803,0.336830,0.332384,0.332100,...,0.000979,0.000772,0.004460,0.163913,0.004006,0.260757,0.160355,0.000902,0.160809,-0.095943
VisualMotor,0.166127,0.281437,0.129792,0.195600,0.132140,0.012664,0.325687,0.325671,0.325460,0.325450,...,-0.004470,-0.004394,-0.003764,0.122073,-0.005622,0.222055,0.125099,-0.000738,0.126957,-0.100720


In [48]:
#save extra tables

#save table with r2 only (not uniq or common part of r2)
pd.concat([df_r2_one, df_r2_two, df_r2_three], axis=1).to_csv(path+'r2_ages_cog_fluid_new.csv')

df_corR_one.to_csv(path+'correlation_r_agepreds_cog_fluid_new.csv')

df_corPval_one.to_csv(path+'correlation_pval_agepreds_cog_fluid_new.csv')

In [49]:
alpha_table = pd.concat([df_alp_two,
                         df_alp_two_age_cogpr,
                         df_alp_two_agepr_cogpr,
                         df_alp_three], axis=1)

display(alpha_table)

alpha_table.to_csv(path+'commonality_cog_fluid_ridge_alphas.csv')

,Alpha2_age_pred+age-orig,Alpha2_age_gap+age-orig,Alpha2_age_pred_cor+age-orig,Alpha2_age_gap_cor+age-orig,Alpha3_2_age-orig+cog-pred,Alpha3_2_age_pred+cog-pred,Alpha3_2_age_gap+cog-pred,Alpha3_2_age_pred_cor+cog-pred,Alpha3_2_age_gap_cor+cog-pred,Alpha3_3_age_pred+age+cog-pred,Alpha3_3_age_gap+age+cog-pred,Alpha3_3_age_pred_cor+age+cog-pred,Alpha3_3_age_gap_cor+age+cog-pred
carit1,10.000000,1.350314,9.047357,4.489251,9.047357,100.000000,18.233480,100.000000,27.213388,18.233480,4.489251,16.496481,8.185467
carit3,5.484417,7.405685,4.961948,11.052951,30.078825,100.000000,54.844166,100.000000,54.844166,33.245979,90.473572,33.245979,44.892513
carit4,6.700188,1.649648,6.700188,5.484417,4.961948,100.000000,7.405685,100.000000,18.233480,10.000000,27.213388,8.185467,10.000000
face1,13.503140,6.061899,12.216773,14.924955,10.000000,100.000000,6.700188,100.000000,11.052951,27.213388,60.618990,20.153377,49.619476
face2,10.000000,14.924955,11.052951,13.503140,11.052951,90.473572,4.961948,90.473572,2.015338,4.961948,40.615860,6.061899,30.078825
face3,5.484417,2.462092,5.484417,4.489251,9.047357,13.503140,3.674662,14.924955,3.324598,4.061586,16.496481,6.700188,16.496481
face4,11.052951,27.213388,9.047357,14.924955,22.275430,100.000000,33.245979,100.000000,49.619476,20.153377,74.056847,27.213388,44.892513
face5,8.185467,4.061586,8.185467,10.000000,14.924955,22.275430,7.405685,22.275430,6.061899,6.700188,24.620924,4.961948,16.496481
face6,5.484417,0.332460,12.216773,16.496481,5.484417,12.216773,3.324598,16.496481,2.462092,2.721339,14.924955,0.100000,8.185467
vism,4.489251,2.227543,5.484417,5.484417,7.405685,40.615860,4.489251,40.615860,2.015338,3.674662,20.153377,3.324598,12.216773
